In [4]:
import requests
from fake_headers import Headers
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def generate_payload(start_date, end_date):
    current_date = datetime.now().strftime("%Y-%m-%d")
    return {
        "CSRFToken": "G1P5qi2QQzdb48cAT4etN5FEw5HKBf3uH2jfE9rWzEI",
        "loadEnd": "0",
        "curTime": current_date,
        "totSeatCntRatioOrder": "",
        "totSeatCntOrder": "",
        "totShowAmtOrder": "",
        "addTotShowAmtOrder": "",
        "totShowCntOrder": "",
        "addTotShowCntOrder": "",
        "dmlMode": "search",
        "startDate": start_date,
        "endDate": end_date,
        "searchType": "",
        "repNationCd": "",
        "wideareaCd": ""
    }

def fetch_data(url, payload, target_movie, period):
    headers = Headers(headers=True).generate()
    response = requests.post(url, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    table_datas = soup.find('tbody', {'id': 'mainTbody'}).find_all('tr')
    results = []

    for row in table_datas:
        movie_name = row.find('td', {'id': 'td_movie'}).get_text(strip=True)
        if movie_name == target_movie:
            if 'findPeriodSeatTicketList' in url:
                ratio = row.find('td', {'id': 'td_totSeatCntRatio'}).get_text(strip=True)
            elif 'findPeriodShowTicketList' in url:
                ratio = row.find('td', {'id': 'td_totDspCntRatio'}).get_text(strip=True)
            elif 'findPeriodScreenTicketList' in url:
                ratio = row.find('td', {'id': 'td_totScrnCntRatio'}).get_text(strip=True)
            results.append((movie_name, ratio, period))
    
    return results

def get_movie_ratios(movie_name, release_date_str):
    release_date = datetime.strptime(release_date_str, "%Y-%m-%d")
    periods = [(0, 7), (7, 14), (14, 21)]

    urls = {
        "seat": "https://kobis.or.kr/kobis/business/stat/boxs/findPeriodSeatTicketList.do",
        "show": "https://kobis.or.kr/kobis/business/stat/boxs/findPeriodShowTicketList.do",
        "screen": "https://kobis.or.kr/kobis/business/stat/boxs/findPeriodScreenTicketList.do"
    }

    results = {
        "seat": [],
        "show": [],
        "screen": []
    }

    for start, end in periods:
        start_date = (release_date + timedelta(days=start)).strftime("%Y-%m-%d")
        end_date = (release_date + timedelta(days=end)).strftime("%Y-%m-%d")
        period = f"{start_date} ~ {end_date}"
        payload = generate_payload(start_date, end_date)

        results["seat"].extend(fetch_data(urls["seat"], payload, movie_name, period))
        results["show"].extend(fetch_data(urls["show"], payload, movie_name, period))
        results["screen"].extend(fetch_data(urls["screen"], payload, movie_name, period))

    return results

# 예제 사용법
movie_name = "서울의 봄"
release_date_str = "2023-11-22"

ratios = get_movie_ratios(movie_name, release_date_str)

print("좌석점유율:")
for entry in ratios["seat"]:
    print(f"기간: {entry[2]}, 영화: {entry[0]}, 좌석점유율: {entry[1]}")

print("\n상영점유율:")
for entry in ratios["show"]:
    print(f"기간: {entry[2]}, 영화: {entry[0]}, 상영점유율: {entry[1]}")

print("\n스크린점유율:")
for entry in ratios["screen"]:
    print(f"기간: {entry[2]}, 영화: {entry[0]}, 스크린점유율: {entry[1]}")


좌석점유율:
기간: 2023-11-22 ~ 2023-11-29, 영화: 서울의 봄, 좌석점유율: 62.4%
기간: 2023-11-29 ~ 2023-12-06, 영화: 서울의 봄, 좌석점유율: 57.6%
기간: 2023-12-06 ~ 2023-12-13, 영화: 서울의 봄, 좌석점유율: 55.1%

상영점유율:
기간: 2023-11-22 ~ 2023-11-29, 영화: 서울의 봄, 상영점유율: 56.5%
기간: 2023-11-29 ~ 2023-12-06, 영화: 서울의 봄, 상영점유율: 52.4%
기간: 2023-12-06 ~ 2023-12-13, 영화: 서울의 봄, 상영점유율: 50.0%

스크린점유율:
기간: 2023-11-22 ~ 2023-11-29, 영화: 서울의 봄, 스크린점유율: 23.7%
기간: 2023-11-29 ~ 2023-12-06, 영화: 서울의 봄, 스크린점유율: 25.0%
기간: 2023-12-06 ~ 2023-12-13, 영화: 서울의 봄, 스크린점유율: 23.6%
